<a href="https://colab.research.google.com/github/NotAndex/Demo/blob/main/Demo_for_Applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Setup

In [ ]:
import os
import datetime

#download yfinance package
!pip install yfinance
import yfinance as yf
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (12, 8)
mpl.rcParams['axes.grid'] = False

# 2 BTC-USD dataset

This dataset contains...

In [ ]:
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365)
symbol = 'BTC-USD'

cc_data = yf.download(symbol, start=start_date, end=today)
cc_data.reset_index(inplace=True,col_fill=('Date'))
cc_data.head()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}$'))
ax1.plot(cc_data['Date'], cc_data['Close'],label = 'Close')
ax1.set_title((f'{symbol} from {start_date} to {today}'), fontsize = 12)
ax1.legend(fontsize=8)

# 3 Data Preprocessing

## 3.1 Data Split

In [32]:
data = cc_data.loc[:,['Close']]

len_data = len(data)
train_data = data[0:int(len_data*0.7)] # 70%
val_data = data[int(len_data*0.7):int(len_data*0.9)] # 20%
test_data = data[int(len_data*0.9):] # 10%


## 3.2 Data Normalization


In [ ]:
train_data_mean = train_data.mean()
train_data_std = train_data.std()

train_data = (train_data - train_data_mean) / train_data_std
val_data = (val_data - train_data_mean) / train_data_std
test_data = (test_data - train_data_mean) / train_data_std